# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import re
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
import pickle

In [2]:
# load data from database
engine = create_engine('sqlite:///disasterResponse.db')
df = pd.read_sql_table('disasterResponse', engine)
X = df['message']
Y = df.loc[:, 'related':'direct_report']

In [13]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Normalize text
    text_nor = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize text
    words = word_tokenize(text_nor)
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    # Reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in words]
    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    # Lemmatize verbs by specifying pos
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline =  Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=6)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fujia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fujia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fujia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=6)

# training
pipeline.fit(X_train, Y_train)

# predicting
Y_to_test = pipeline.predict(X_test)

D:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
Y_to_test_df = pd.DataFrame(Y_to_test)
Y_to_test_df.columns = Y_test.columns
for name in list(Y_test.columns):
    print(name, 'test : \n', classification_report(Y_test.loc[:, name], Y_to_test_df.loc[:, name]))

related test : 
               precision    recall  f1-score   support

           0       0.61      0.47      0.53      1205
           1       0.85      0.90      0.87      3999
           2       0.33      0.42      0.37        40

   micro avg       0.80      0.80      0.80      5244
   macro avg       0.59      0.60      0.59      5244
weighted avg       0.79      0.80      0.79      5244

request test : 
               precision    recall  f1-score   support

           0       0.90      0.97      0.93      4396
           1       0.74      0.42      0.53       848

   micro avg       0.88      0.88      0.88      5244
   macro avg       0.82      0.69      0.73      5244
weighted avg       0.87      0.88      0.87      5244

offer test : 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      5216
           1       0.00      0.00      0.00        28

   micro avg       0.99      0.99      0.99      5244
   macro avg       0.50  

D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x000001E4CCD6E598>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
               oob_score=Fal

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),
    'clf__estimator__n_estimators': (100, 120, 140)
    
}


cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=8)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
cv.fit(X_train, Y_train)
Y_to_test = cv.predict(X_test)
Y_to_test_df = pd.DataFrame(Y_to_test)
Y_to_test_df.columns = Y_test.columns
for name in list(Y_test.columns):
    print(name, 'test : \n', classification_report(Y_test.loc[:, name], Y_to_test_df.loc[:, name]))

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


related test : 
               precision    recall  f1-score   support

           0       0.63      0.44      0.52      1205
           1       0.84      0.92      0.88      3999
           2       0.41      0.40      0.41        40

   micro avg       0.81      0.81      0.81      5244
   macro avg       0.63      0.59      0.60      5244
weighted avg       0.79      0.81      0.79      5244

request test : 
               precision    recall  f1-score   support

           0       0.91      0.98      0.94      4396
           1       0.80      0.48      0.60       848

   micro avg       0.90      0.90      0.90      5244
   macro avg       0.86      0.73      0.77      5244
weighted avg       0.89      0.90      0.89      5244

offer test : 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      5216
           1       0.00      0.00      0.00        28

   micro avg       0.99      0.99      0.99      5244
   macro avg       0.50  

D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


fire test : 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      5186
           1       0.00      0.00      0.00        58

   micro avg       0.99      0.99      0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.98      0.99      0.98      5244

earthquake test : 
               precision    recall  f1-score   support

           0       0.97      0.99      0.98      4742
           1       0.89      0.73      0.80       502

   micro avg       0.97      0.97      0.97      5244
   macro avg       0.93      0.86      0.89      5244
weighted avg       0.96      0.97      0.96      5244

cold test : 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      5121
           1       1.00      0.03      0.06       123

   micro avg       0.98      0.98      0.98      5244
   macro avg       0.99      0.52      0.53      5244
weighted avg       0.98   

NameError: name 'Y_pred' is not defined

In [11]:
print("optimal model: ", cv.best_params_)

optimal model:  {'clf__estimator__n_estimators': 140, 'vect__ngram_range': (1, 2)}


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [22]:
pipeline2 =  Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=6)))
    ])
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x000001E4CCD6E598>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=6),
              n_jobs=None))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
    

In [21]:
Y_train.tail()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
8527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4714,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10196,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
8419,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19145,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# training
pipeline2.fit(X_train, Y_train)

# predicting
Y_to_test2 = pipeline2.predict(X_test)
Y_to_test_df2 = pd.DataFrame(Y_to_test2)
Y_to_test_df2.columns = Y_test.columns
for name in list(Y_test.columns):
    print(name, 'test2 : \n', classification_report(Y_test.loc[:, name], Y_to_test_df2.loc[:, name]))

related test2 : 
               precision    recall  f1-score   support

           0       0.61      0.16      0.25      1205
           1       0.79      0.97      0.87      3999
           2       0.32      0.15      0.20        40

   micro avg       0.78      0.78      0.78      5244
   macro avg       0.57      0.43      0.44      5244
weighted avg       0.74      0.78      0.72      5244

request test2 : 
               precision    recall  f1-score   support

           0       0.91      0.96      0.94      4396
           1       0.72      0.51      0.60       848

   micro avg       0.89      0.89      0.89      5244
   macro avg       0.82      0.74      0.77      5244
weighted avg       0.88      0.89      0.88      5244

offer test2 : 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      5216
           1       0.00      0.00      0.00        28

   micro avg       0.99      0.99      0.99      5244
   macro avg       0.5

cold test2 : 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      5121
           1       0.71      0.38      0.50       123

   micro avg       0.98      0.98      0.98      5244
   macro avg       0.85      0.69      0.74      5244
weighted avg       0.98      0.98      0.98      5244

other_weather test2 : 
               precision    recall  f1-score   support

           0       0.96      0.99      0.97      4969
           1       0.53      0.20      0.29       275

   micro avg       0.95      0.95      0.95      5244
   macro avg       0.75      0.60      0.63      5244
weighted avg       0.94      0.95      0.94      5244

direct_report test2 : 
               precision    recall  f1-score   support

           0       0.87      0.96      0.91      4238
           1       0.67      0.38      0.48      1006

   micro avg       0.85      0.85      0.85      5244
   macro avg       0.77      0.67      0.70      5244
weighted av

In [31]:
parameters = {
    'vect__ngram_range': ((1, 2),(1, 3)),
    'clf__estimator__learning_rate': (1.0, 0.5),
    'clf__estimator__n_estimators': (80, 120, 160)
    
}


cv2 = GridSearchCV(pipeline2, param_grid=parameters, n_jobs=8)

In [32]:
cv2.fit(X_train, Y_train)
# predicting
Y_to_test2 = pipeline2.predict(X_test)
Y_to_test_df2 = pd.DataFrame(Y_to_test2)
Y_to_test_df2.columns = Y_test.columns
for name in list(Y_test.columns):
    print(name, 'test2 : \n', classification_report(Y_test.loc[:, name], Y_to_test_df2.loc[:, name]))

related test2 : 
               precision    recall  f1-score   support

           0       0.61      0.16      0.25      1205
           1       0.79      0.97      0.87      3999
           2       0.32      0.15      0.20        40

   micro avg       0.78      0.78      0.78      5244
   macro avg       0.57      0.43      0.44      5244
weighted avg       0.74      0.78      0.72      5244

request test2 : 
               precision    recall  f1-score   support

           0       0.91      0.96      0.94      4396
           1       0.72      0.51      0.60       848

   micro avg       0.89      0.89      0.89      5244
   macro avg       0.82      0.74      0.77      5244
weighted avg       0.88      0.89      0.88      5244

offer test2 : 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      5216
           1       0.00      0.00      0.00        28

   micro avg       0.99      0.99      0.99      5244
   macro avg       0.5

### 9. Export your model as a pickle file

In [ ]:
file = open('ML_Pipline.pkl', 'wb')
pickle.dump(cv2, open(file, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.